# Preprocessing the Data
Part of BA Thesis by Enis Settouf
<br>
[Source:](https://de.openlegaldata.io/pages/api/) Data used by OpenLegalData.io:
DSource ata used by OpenLegalData.io:


Please set the Runtime-shape to high RAM (Runtime - change runtime type)

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# 1. Preparing Runtime

Installing libraries:

In [ ]:
! pip install html2text
! pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Mounting Google Drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# -*- coding: iso-8859-1 -*-
import json
from html2text import html2text
from tqdm.auto import tqdm
import time
from google.colab import files
import re
from sklearn.model_selection import train_test_split

Opening files and splitting for reduction:

In [ ]:
print("--- Process started ---")
start_time = time.time()

with open('/content/drive/MyDrive/ba-thesis/data-pre-processing/data/cases.jsonl', 'r') as jsonl_file:
    json_list = list(jsonl_file)

with open('/content/drive/MyDrive/ba-thesis/data-pre-processing/data/laws.jsonl', 'r') as jsonl_file:
    json_list.extend(list(jsonl_file))

jsonl_file.close()

# Shuffle list after adding laws to cases list
import random
random.shuffle(json_list)

# Splitting the list to make a smaller set for faster training
shrink_factor = 0.01
print(len(json_list))
json_list = json_list[0:int(round((len(json_list)*shrink_factor),0))]
print(len(json_list))

# Split json list into train and test sets
json_train, json_test = train_test_split(json_list, test_size=0.2, shuffle=True, random_state=2)

id = 0
file_count = 0

--- Process started ---
259017
2590


## 2. Download and initialize spacy package

Spacy is initialized and the German package is downloaded.

In [ ]:
from __future__ import unicode_literals, print_function
from spacy.lang.de import German

import spacy
spacy.cli.download("de_core_news_sm")

nlp = German()
nlp.add_pipe('sentencizer')
nlp.max_length = 6000000 # instead of 1,000,000; max length was about 5,000,000

✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


Special legal acronyms and abbreviations are specified. Spacy will now identify them as nouns and won't end the sentence after the puncuation mark. 

In [ ]:
# Add special case rule
from spacy.symbols import ORTH

special_cases = [
    'Rdnr.', 'm.w.N.', 'Tz.', 'S.p.A.',
    'A.', 'a.', 'AA', 'AbgG', 'ABl.', 'abl.', 'Abs.', 'abs.', 'Abschn.', 'Abse.', 'AbzG', 'abzgl.', 'AcP', 'a.D.', 'AEG', 'AEntG', 'AFG', 'AFRG', 'AG', 'AGB', 'AGBG', 'AGL', 'ähnl.', 'AK(-Bearb.)', 'AktG', 'AKW', 'allg.', 'allg. A.', 'allgem.', 'allgem. A.', 'ALR', 'Alt.', 'AmtlAnz', 'AmtsBl.', 'AN', 'and.', 'angef.', 'Anl.', 'Anm.', 'AO', 'AOK', 'AP', 'ArbG', 'ArbGG', 'ArbPlSchG', 'ArbSchG', 'ArbZG', 'ArbZRG', 'ArchBürgR', 'ARD', 'arg. e(x)', 'ARGE', 'ARSP', 'Art.', 'AStA', 'AT', 'AtomG', 'AU', 'aufgeh.', 'Aufl.', 'AÜG', 'ausf.', 'AusfG', 'AusfVO', 'AuslG', 'Ausn.', 'AVG', 'AVO', 'Azubi', 'B', 'BAG', 'BAGE', 'BAnz.', 'BArbBl.', 'BauGB', 'BayJMBl.', 'BayObLG', 'BayObLGSt', 'BayObLGZ', 'BayRPfl', 'BBeamtenG', 
    'BBesG', 'BB', 'BBG', 'BBiG', 'Bd.', 'Bde.', 'Bdg.', 'BDSG', 'BDZV', 'BeamtenRRG', 'BeamtVG', 'Bearb.', 'begr.', 'Beil.', 'Bek.', 'ber.', 'BerGer', 'BErzGG', 'bes.', 'BeschFG', 'Beschl.', 'best.', 'bestr.', 'Betr.', 'betr.', 'BetrAVG', 'BetrVG', 'BeurkG', 'Bf.', 'BfA', 'BfB', 'BFH', 'BGB', 'BGBl.', 'BGH', 'BGH NJW', 'BGHSt', 'BGHZ', 'BGSG', 'BImSchG', 'BinnSchG', 'BiRiLiG', 'BJagdG', 'BKA', 'Bl.', 'BMAuS', 'BMELF', 'BMF', 'BMI', 'BMin', 'BMJ', 'BMU', 'BMW', 'BNotO', 'BO', 'BPräs.', 'BR', 'BRAGO', 'BRAO', 'BRD', 'BReg.', 'BRep', 'BRRG', 'BSeuchenG', 'BSG', 'BSGE', 'BSHG', 'BSozHilfeG', 'Bsp.', 'Bst.', 'BStBl.', 'BT', 'BT-Drucks.', 'BtG', 'BtMG', 'BtMVV', 'BUrlG', 'BVerfG', 'BVerfGE', 'BVerfGG', 'BVerwG', 'BVerwGE', 'BVwZG', 'BWG', 'BWO', 'BZRG', 'bzgl.', 'bzw.', 'CCC', 'CDU', 'CIC', 'CR', 'CSU', 'D', 'DAR', 'dar.', 'DB', 'DDR', 'DEFA', 'ders.', 'DGB', 'dies.', 'DIHT', 'DIN', 'Diss.', 'div.', 'DJT', 'DJZ', 'DKP', 'DM', 'DNotZ', 'DO', 'DÖV', 'DR', 'Dr.', 'Dr. h.c.', 'Dr. med.', 'Dr. oec.', 'Dr. oec. publ.', 'Dr. pharm.', 'Dr. phil.', 'Dr. rer. nat.', 'Dr. rer. oec.', 'Dr. rer. pol.', 'Dr. theol.', 'DRiG', 'DRiZ', 'Drucks.', 'DStR', 'dto.', 'DtZ', 'DV', 'DVBl.', 'DVO', 'E', 'EALR', 'ebd.', 'EBRG', 'EBV', 'EDV', 'EEF', 'EFZG', 'EG', 'eG', 'EGBGB', 'EGHGB', 'EGKStV', 'EGL', 'eGmbH', 'eGmuH', 'EGStGB', 'EGV', 'EheG', 'EheSchlRG', 'EinfG', 'eingef.', 'EinigV', 'Einl.', 'einschl.', 'EMRK', 'ErbbauRVO', 'ErbStG', 'Erg.', 'erk. Sen.', 'Erl.', 'ESJ', 'EStG', 'etc.', 'EuGH', 'EuGHE', 'Euratom', 'EuratomGV', 'EUR', 'Euro EG', 'EUV', 'EuZW', 'EV', 'EVertr.', 'evtl.', 'EWGV', 'EWIV', 'EZB', 'EzSt', 'F.', 'f.', 'FA', 'Fa.', 'FamRZ', 'FAO', 'FDP', 'Festschr.', 'ff.', 'FG', 'FGG', 'FGO', 'FH', 'FHSR', 'FL', 'fl', 'FN', 'Fn.', 'form.', 'FOSR', 'fr.', 'FS', 'FSK', 'FStrG', 'FU', 'FVG', 'Fz.', 'G', 'GA', 'GastG', 'GATT', 'GBl.', 'GBO', 'GbR', 'geänd.', 'GebrMG', 'Gedschr.', 'geg.', 'gem.', 'GemSOBG', 'GenG', 'GenTG', 'Ges.', 'GeschmMG', 'GeschO', 'GeschOBR', 'GeschOBT', 'GewO', 'GewStG', 'GG', 'gg.', 'ggf.', 'GGO', 'ggü.', 'ggüb.', 'GjSM', 'GK', 'GKG', 'Gl.', 'GmbH', 'GmbHG', 'GMBl.', 'GmS', 'GO', 'GO BR', 'GO BReg', 'GO BT', 'Grds.', 'grdsl.', 'GRG', 'GrS', 'GS', 'GÜG', 'GüKG', 'GUS', 'GVBl.', 'GVG', 'GV NW',
    'GVOBl.', 'GWB', 'HaftPflG', 'HAG', 'Halbs.', 'HandwO', 'HausratsVO', 'HaustürWG', 'h.c.', 'Hdlg.', 'Hess.', 'HESt', 'heut.', 'heut. Rspr.', 'HGB', 'hins.', 'HRefG', 'HRG', 'HRR', 'Hrsg.', 'HWiG', 'i.A.', 'IAEA', 'ib.', 'ibd.', 'ibid.', 'IHK', 'i.K.', 'incl.', 'inkl.', 'inkl. MwSt./ Mwst.', 'insb.', 'insbes.', 'InsO', 'Internet', 'i.O.', 'IPR', 'IQ', 'i.R.', 'IWF', 'i.W.', 'i.Wes.', 'i. Zw.', 'JA', 'Jahrb.', 'JArbSchG', 'JGG', 'Jh.', 'JherJb', 'JMBl.', 'JMBl. NW', 'JP', 'JPA', 'JR', 'JSchG', 'JurA', 'Jura', 'JurJB', 'JuS', 'JVA', 'JW', 'JZ', 'Kfz', 'kzfr.', 'KG', 'KJHG', 'KO', 'KostO', 'KP', 'KPD', 'KR', 'Kripo', 'KrWG', 'KSchG', 'KStG', 'KTS', 'KunstUrhG', 'KV', 'KWG', 'KWKG', 'LadSchlG', 'LAG', 'LFZG', 'LG', 'lgfr.', 'Lit.', 'LK', 'LK(-Bearb.)', 'LKA', 'LM', 'LMBG', 'LMin', 'LohnfortzahlungsG', 'LPresseG', 'LSA', 'LSG', 'LStVG', 'lt.', 'LuftVG', 'LuftVO', 'LuftVZO', 'LVA', 'LVerfG', 'LVerfGE', 'LVerfGG', 'LWahlG', 'LZ', 'LZG', 'm. Änd.', 'MarkenG', 'MABl.', 'mat.', 'mbH', 'MdB', 'MdL', 'MDR', 'ME', 'm.E.', 'MedR', 'MEPolG', 'MfOR', 'MfS', 'mgl.', 'Mglkt.', 'MHG', 'MinBl.', 'MitbestG', 'Mod.', 'mögl.', 'Mot.', 'MRK', 'MRRG', 'MS', 'MschrKrim', 'MüKo(-Bearb.)', 'MünchKomm(-Bearb.)', 'MuSchG', 'm.zust.Anm.', 'NachhBG', 'Nachw.', 'Nachw. b.', 'NachwG', 'NATO', 'Nds.', 'NdsRpfl', 'Neuf.', 'neugef.', 'NJ', 'NJW', 'NJW-RR', 'Nr.', 'NRW', 'NSDAP', 'NStE', 'NStZ', 'NW', 'NZA', 'NZV', 'NZVwR', 'o.', 'OAS', 'OAE', 'OAU', 'OB', 'OBM', 'od.', 'OECD', 'OER', 'öff.', 'OGH', 'OGHSt', 'OGHZ', 'OHG', 'ÖJZ', 'OLG', 'OLGSt', 'OLGZ', 'OPEC', 'OrgKG', 'OVG', 'OWiG', 'OWVO', 'PAG', 'PartG', 'PartGG', 'PaßG', 'PatG', 'PBefG', 'PflegeVG', 'PflVersG', 'PflVG', 'PH', 'PIN', 'PolG', 'PolVO', 'Postf.', 'PostG', 'pp.', 'PresseG', 'PreußOVG', 'ProstG', 'PrStGB', 'ProdHaftG', 'ProdSG', 'Prof.', 'Prot.', 'PrOVG', 'PStG', 'PrStGB', 'PublKG', 'q.e.d.', 'RA', 'RabelsZ', 'RAG', 'RAin', 'RBerG', 'RdErl.', 'Rdnr.', 'RdSchr.', 'rel.', 'RF', 'RG', 'RGBl.', 'RGRK', 'RGSt', 'RGZ', 'Rili', 'RIW', 'RL', 'RPflG', 'RRG', 'RsprEinhG', 'RStGB', 'RuStAG', 'RV', 'RVO', 'RW', 'rw', 'S.', 's.', 'SchE', 'SchG', 'SchlHA', 'SchlHOLG', 'SCE', 'ScheckG', 'SchiffsRG', 'Sch-Sch(-Bearb.)', 'Schr.', 'SchwarbG', 'SchwbG', 'SchwKG', 'SchwZStr', 'scil.', 'SE', 'SED', 'SeemG', 'Sen.', 'SeuffA', 'SG', 'SGB', 'SGG', 'SHG', 'SiG', 'SiGS', 'SiN', 'sinngem.', 'SiÜ', 'SiZess.', 'SJZ', 'SK(-Bearb.)', 's.o.', 'sog.', 'SoldG', 'Sonderbeil.', 'SPD', 'SprAuG', 'SpuRt', 'SS', 'StA', 'StaatsGH', 'StaatsGHG', 'StÄG', 'stG', 'StGB', 'StHG', 'Stpfl.', 'StPO', 'str.', 'StrÄndG', 'StrG', 'StrRG', 'StrVert', 'StrWG', 'stud. iur.', 'StVG', 'StVO', 'StVollstrO', 'StVollzÄndG', 'StVollzG', 'StVZO', 's.u.', 'SubvG', 'SV', 'SVG', 'TA', 'TA-Lärm', 'TA-Luft', 'TAN', 'TB', 'TdL', 'teilw.', 'TH', 'Thür.', 'TKG', 'TO. A', 'TO. B', 'TPG', 'TRG', 'TRIPS', 'TV', 'TVG', 'tw.', 'Tz.', 'TzWrG', 'u.', 'UdSSR', 'UmstG', 'umstr.', 'UmweltHG', 'UmwG', 'UmwStG', 'UN', 'unmgl.', 'Unmglkt.', 'unmögl.', 'UNO', 
    'UrheberRG', 'UrhG', 'USA', 'UStG', 'usw.', 'UVEG', 'UWG', 'UZwG', 'v.', 'VA', 'VAG', 'VAHRG', 'Var.', 'VerbrKrG', 'VereinsG', 'VerfGH', 'VerfGHG', 'VerfSchG', 'VermBG', 'VermG', 'VerschG', 'VersG', 'VersR', 'VG', 'VGH', 'vgl.', 'Vgl.', 'VHTr', 'VO(en)', 
    'VOB', 'VOBl.', 'VOR', 'Vor.', 'VP', 'VRS', 'VS', 'vs', 'VuR', 'VVaG', 'VVG', 'VwGO', 'VwKostG', 'VwVfG', 'VwVG', 'VwZG', 'WA', 'WaffG', 'WahlO', 'Warn.', 'WaStrG', 'WE', 'WEG', 'weg.', 'WEU', 'WG', 'WGO-MfOR', 'wg.', 'WHG', 'WHO', 'WiKG', 'WiRO', 'WiStG', 'wistra', 'WM', 'WoBauG', 'WpHG', 'WPM', 'WRV', 'WS', 'WStG', 'WTO', 'WV', 'w.z.b.w.', 'WZG', 'ZAkDR', 'ZDG', 'ZEuP', 'ZG', 'ZGB', 'ZGR', 'z. Hd.', 'ZHR', 'Ziff.', 'ZIP', 'ZPO', 'ZRP', 'ZS', 'ZSEG', 'ZStW', 'zust.', 'zust.Anm.', 'ZustVO', 'zw.', 'ZVG', 'ZVS', 'zzgl.', 'ZZP', 'a.A.', 'aA', 'abs. h.A.', 'abs. hA', 'a.E.', 'aE', 'a.F.', 'aF', 'a.l.i.c.', 'alic', 'bish. Rspr.', 'bisher. Rspr.', 'c.i.c.', 'cic', 'c.s.q.n.', 'csqn', 'Ct.', 'Ct', 'd.h.', 'dh', 'Dr. iur.', 'Dr. jur.', 'E.G.', 'EG', 'EGmbH', 'EGmuH', '', 'E.U.', 'EU', 'e.V.', 'eV', 'E.W.G.', 'EWG', 'FBA', 'FbA', 'fr. Rspr.', 'frRspr', 'ganz h.A.', 'ganz hA', 'G.o.A.', 'GoA', 'h.A.', 'hA', 'h.L.', 'hL', 'h.M.', 'hM', 'h.Rspr.', 'hRspr', 'Hs.', 'Hs', 'i.d.F.', 'idF', 'i.d.S.', 'idS', 'i.e.', 'ie', 'i.H.d.', 'iHd', 'i.H.v.', 'iHv', 'i.R.d.', 'iRd', 'i.S.d.', 'iSd', 'i.S.v.', 'iSv', 'i.V.', 'iV', 'i.w.S.', 'iwS', 'KGaA', 'KG a.A.', 'Lb.', 'Lb', 'LKW', 'Lkw', 'M.A.', 'MA', 'm.a.W.', 'maW', 'Mio.', 'Mio', 'M.M.', 'MM', 'm.w.N.', 'mwN', 'm.W.v.', 'mWv', 'n.F.', 'nF', 'o.Ä.', 'oÄ', 'o.g.', 'og', 'p.F.V.', 'pFv', 'PKW', 'Pkw', 'ppA.', 'ppa.', 'p.V.', 'pV', 'Rdn.', 'Rdn', 'RN', 'Rn.', 'Rn', 'Rspr.', 'Rspr', 'Rz.', 'Rz', 's.a.', 'sa', 'Sch/Sch(-Bearb.)', '', 
    'st. Rspr.', 'stRspr', 'u.a.', 'ua', 'u.U.', 'uU', 'V.', 'V', 'W.G.G.', 'WGG', 'z.B.', 'zB', 'z.T.', 'zT', 'z.Z.', 'zZ', 'a.a.O.', 'aaO', 'abs. h.M.', 'abs. hM', 'c.p.c.', 'cpc', 'ganz h.M.', 'ganz hM', 'h.Lit.', 'hLit', 'HS.', 'HS', 'i.d.R.', 'idR', 'i.e.S.', 'ieS', 'i.S.', 'iS', 'i.ü.', 'iü', 'm.N.', 'mN', 'MwSt.', 'Mwst.', 'p.V.V.', 'pVV', 'u.a.m.', 'uam', 'i.E.', 'iE', 'i.S.e.', 'iSe', 'i.V.m.', 'iVm'
]

for case in special_cases:
    special_case = [{ORTH: case}]
    nlp.tokenizer.add_special_case(case, special_case)

# 3. Process texts and save in files 

Two seperate cells because of RAM issues

2.1 Validation Set

In [ ]:
file_sentences_test = open('/content/drive/MyDrive/ba-thesis/data-pre-processing/data_final/data_sentences_validation_1p.csv', 'w', encoding="utf-8")
file_fulldocs_test = open('/content/drive/MyDrive/ba-thesis/data-pre-processing/data_final/data_fulldocs_validation_1p.csv', 'w', encoding="utf-8")

for json_doc_test in tqdm(json_test):
    
    dict_test = json.loads(json_doc_test)
    content_test = html2text(dict_test['content'])
    content_test = content_test.strip()
    content_test = content_test.replace('\n', ' ')
    content_test = content_test.replace("## Tenor  ", "")
    content_test = re.sub(' +', ' ', content_test)

    if content_test != '' and not content_test.isspace():
        file_fulldocs_test.write(str(id) + '_adelimiter528_' + content_test + '_adelimiter528_' + '\n')
        
        # Split text into sentences
        doc = nlp(content_test)
        split_sentences = list(doc.sents)

        for sent in split_sentences:
            file_sentences_test.write(str(id) + '_adelimiter528_' + str(sent) + '_adelimiter528_' + '\n')
    
    id += 1

    if id % 50000 == 0:
        print("--- Intermediate time check: %s minutes ---" % ( (time.time() - start_time) / 60 ))

file_sentences_test.close()
file_fulldocs_test.close()

print("--- Process ended: %s minutes ---" % ( (time.time() - start_time) / 60 ))

  0%|          | 0/518 [00:00<?, ?it/s]

--- Process ended: 1.3020344932874044 minutes ---


2.2 Train Set

In [ ]:
file_sentences_train = open('/content/drive/MyDrive/ba-thesis/data-pre-processing/data_final/data_sentences_train_1p.csv', 'w', encoding="utf-8")
file_fulldocs_train = open('/content/drive/MyDrive/ba-thesis/data-pre-processing/data_final/data_fulldocs_train_1p.csv', 'w', encoding="utf-8")

id = 0
for json_doc_train in tqdm(json_train):
    
    dict_train = json.loads(json_doc_train)
    content_train = html2text(dict_train['content'])
    content_train = content_train.strip()
    content_train = content_train.replace('\n', ' ')
    content_train = content_train.replace("## Tenor  ", "")
    content_train = re.sub(' +', ' ', content_train)
        
    # Save only strings that are not blank
    if content_train != '' and not content_train.isspace():
        file_fulldocs_train.write(str(id) + '_adelimiter528_' + content_train + '_adelimiter528_' + '\n')

        # Split text into sentences
        doc = nlp(content_train)
        split_sentences = list(doc.sents)

        for sent in split_sentences:
            file_sentences_train.write(str(id) + '_adelimiter528_' + str(sent) + '_adelimiter528_' + '\n')
    
    id += 1

    if id % 5000 == 0:
        print("--- Intermediate time check: %s minutes ---" % ( (time.time() - start_time) / 60 ))

file_sentences_train.close()
file_fulldocs_train.close()

print("--- Process ended: %s minutes ---" % ( (time.time() - start_time) / 60 ))

  0%|          | 0/2072 [00:00<?, ?it/s]

--- Process ended: 2.4488534847895305 minutes ---


Sending an email Notification:

In [ ]:
# import smtplib

# server = smtplib.SMTP('smtp.gmail.com', 587)
# server.starttls()
# server.login("email_adress", "password")

# msg = "The model is pushed to the hub."
# server.sendmail("email_adress", msg)
# server.quit()

(221,
 b'2.0.0 closing connection f22-20020a056638113600b00349bb70ab9fsm1442574jar.152 - gsmtp')